## Модуль геопланирования и кластеризации точек
### Описание проекта

Данный ноутбук представляет собой прототип модуля геопланирования, предназначенного для автоматического распределения географических точек по дням месяца, их кластеризации в дневные маршруты и последующей визуализации результатов.

Решение ориентировано на практическое использование в задачах полевого планирования (менеджеры, аудиторы, мерчандайзеры) и служит базой для дальнейшего выноса логики в отдельные Python-модули (clustering, routing, utils, visualize).

<b>Ключевые принципы:</b>
* воспроизводимость результатов;
* конфигурируемые ограничения (дни, размеры кластеров);
* устойчивость к неполным данным;
* бесплатный стек инструментов.

### Конфигурация: настройки проекта

In [1]:
# Загрузка данных
DATA_PATH = "C:/Users/persc/project/src/data/points.csv"

# Период планирования
YEAR = 2025
MONTH = 3

# Менеджеры
N_MANAGERS = 3
MANAGER_IDS = list(range(1, N_MANAGERS + 1))

# Ограничения
MAX_POINTS_PER_DAY = 12
MIN_POINTS_PER_DAY = 8
MAX_TRAVEL_TIME_HOURS = 4
WORKDAY_HOURS = 8

# Обслуживание
DEFAULT_SERVICE_TIME_MIN = 30

# География
OFFICE_ADDRESS = "Нижний Новгород, Большая Покровская, 1"

# Алгоритмы
RANDOM_SEED = 42

# Визуализация
VIZUALIZATION_DATE = '2025-03-04'

### Импорт функций

In [2]:
import pandas as pd
import numpy as np
import requests
import folium
import time

from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from workalendar.europe import Russia
from sklearn.cluster import KMeans

from ortools.constraint_solver import pywrapcp, routing_enums_pb2

load C:\Users\persc\venv-routing\lib\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\persc\venv-routing\lib\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\persc\venv-routing\lib\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\persc\venv-routing\lib\site-packages\ortools\.libs\re2.dll...
load C:\Users\persc\venv-routing\lib\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\persc\venv-routing\lib\site-packages\ortools\.libs\highs.dll...
load C:\Users\persc\venv-routing\lib\site-packages\ortools\.libs\ortools.dll...


### utils
### Проверка и подготовка входных данных

In [3]:
def geocode_address(address: str):
    """Преобразует адрес в координаты (lat, lon)."""
    geo = Nominatim(user_agent="geoplanning")
    loc = geo.geocode(address)
    return loc.latitude, loc.longitude


def validate_points(df: pd.DataFrame) -> pd.DataFrame:
    """
    Проверяет входные данные.
    """
    required = ['point_id', 'latitude', 'longitude', 'visits_per_month']
    missing = set(required) - set(df.columns)
    if missing:
        raise ValueError(f"Отсутствуют обязательные столбцы: {missing}")

    df = df.dropna(subset=['latitude', 'longitude'])
    df = df[(df.latitude.between(-90, 90)) & (df.longitude.between(-180, 180))]
    df = df[df.visits_per_month > 0]

    if 'service_time_min' not in df.columns:
        df['service_time_min'] = DEFAULT_SERVICE_TIME_MIN
    else:
        df['service_time_min'] = df['service_time_min'].fillna(DEFAULT_SERVICE_TIME_MIN)

    return df


def calculate_distance(p1, p2) -> float:
    """Расстояние между двумя точками в километрах."""
    return geodesic(p1, p2).km


def get_working_days(year: int, month: int):
    """Список рабочих дней месяца (5/2, РФ)."""
    cal = Russia()
    days = pd.date_range(f"{year}-{month}-01", f"{year}-{month}-28")
    return [d for d in days if cal.is_working_day(d)]


def yandex_route_link(coords):
    """Ссылка на маршрут в Яндекс Картах."""
    points = "~".join([f"{lat},{lon}" for lat, lon in coords])
    return f"https://yandex.ru/maps/?rtext={points}&rtt=auto"

### clustering
### Кластеризация точек внутри дня

Выбран алгоритм KMeans, так как он:
* прост в интерпретации;
* даёт воспроизводимый результат при фиксированном random_state;
* позволяет автоматически вычислять число кластеров.

In [4]:
def expand_visits_flat(df: pd.DataFrame) -> pd.DataFrame:
    """
    Разворачивает visits_per_month в плоский список визитов
    БЕЗ привязки к датам.
    """
    rows = []

    for _, r in df.iterrows():
        for _ in range(int(r.visits_per_month)):
            rows.append({
                'point_id': r.point_id,
                'manager_id': r.manager_id,
                'latitude': r.latitude,
                'longitude': r.longitude,
                'service_time_min': r.service_time_min
            })

    return pd.DataFrame(rows)

def assign_visits_to_days(
    visits: pd.DataFrame,
    work_days: list,
    max_points_per_day: int
):
    """
    Формирует реальный календарь:
    - 1 менеджер
    - 1 день
    - ≤ max_points_per_day
    Возвращает:
    - schedule (что обслуживаем)
    - unserved (что не влезло в месяц)
    """
    schedule_rows = []
    unserved_rows = []

    for manager_id, grp in visits.groupby('manager_id'):
        # Можно отсортировать — например, по расстоянию от офиса
        grp = grp.reset_index(drop=True)

        day_idx = 0

        for i in range(0, len(grp), max_points_per_day):
            chunk = grp.iloc[i:i + max_points_per_day]

            if day_idx >= len(work_days):
                unserved_rows.append(chunk)
            else:
                day = work_days[day_idx]
                for _, r in chunk.iterrows():
                    schedule_rows.append({
                        'point_id': r.point_id,
                        'manager_id': manager_id,
                        'visit_day': day,
                        'latitude': r.latitude,
                        'longitude': r.longitude,
                        'service_time_min': r.service_time_min
                    })
                day_idx += 1

    schedule = pd.DataFrame(schedule_rows)
    unserved = (
        pd.concat(unserved_rows)
        if unserved_rows else
        pd.DataFrame(columns=visits.columns)
    )

    return schedule, unserved


def cluster_points(points: pd.DataFrame, max_points: int, random_state: int = 42):
    """
    Кластеризация точек одного дня.
    Размер кластера не превышает max_points.
    """
    if len(points) <= max_points:
        points['cluster_id'] = 0
        return points

    n_clusters = int(np.ceil(len(points) / max_points))
    model = KMeans(n_clusters=n_clusters, random_state=random_state)
    points = points.copy()
    points['cluster_id'] = model.fit_predict(points[['latitude', 'longitude']])
    return points

def assign_managers(df: pd.DataFrame, n_managers: int, random_state: int = 42):
    """
    Закрепляет точки за менеджерами на основе географии.
    Используется KMeans по координатам.
    """
    model = KMeans(n_clusters=n_managers, random_state=random_state)
    df = df.copy()
    df['manager_id'] = model.fit_predict(df[['latitude', 'longitude']]) + 1
    return df

### routing
### Построение маршрута внутри кластера

In [5]:
def osrm_route_duration(coords, sleep_sec=0.5):
    """
    Возвращает длительность маршрута в секундах.
    При ошибке возвращает None.
    """
    if len(coords) < 2:
        return 0

    coord_str = ';'.join([f"{lon},{lat}" for lat, lon in coords])
    url = f"http://router.project-osrm.org/route/v1/driving/{coord_str}"
    params = {"overview": "false"}

    try:
        r = requests.get(url, params=params, timeout=10)

        # если сервер ответил не 200
        if r.status_code != 200:
            return None

        data = r.json()

        if 'routes' not in data or not data['routes']:
            return None

        time.sleep(sleep_sec)  # защита от rate limit
        return data['routes'][0]['duration']

    except Exception:
        return None


def build_route(points: pd.DataFrame, office_coord):
    """
    Строит маршрут внутри кластера (TSP).
    Возвращает точки с order_in_route.
    """
    if len(points) == 1:
        points = points.copy()
        points['order_in_route'] = 1
        return points

    coords = [office_coord] + list(zip(points.latitude, points.longitude))
    coord_str = ';'.join([f"{lon},{lat}" for lat, lon in coords])
    url = f"http://router.project-osrm.org/table/v1/driving/{coord_str}?annotations=duration"
    matrix = requests.get(url).json()['durations']

    manager = pywrapcp.RoutingIndexManager(len(coords), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def time_cb(i, j):
        return int(matrix[manager.IndexToNode(i)][manager.IndexToNode(j)])

    transit_cb = routing.RegisterTransitCallback(time_cb)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_cb)

    search = pywrapcp.DefaultRoutingSearchParameters()
    search.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    solution = routing.SolveWithParameters(search)

    if solution is None:
        points['order_in_route'] = None
        return points

    order = []
    idx = routing.Start(0)
    while not routing.IsEnd(idx):
        node = manager.IndexToNode(idx)
        if node != 0:
            order.append(node - 1)
        idx = solution.Value(routing.NextVar(idx))

    ordered = points.iloc[order].copy()
    ordered['order_in_route'] = range(1, len(ordered) + 1)
    return ordered

### visualize
### Визуализация кластеров и маршрутов

In [6]:
def visualize_day(schedule_df, points_df, office_coord, selected_day):
    """Визуализация маршрутов всех менеджеров за выбранный день."""
    day_df = schedule_df[schedule_df.visit_day == selected_day]

    m = folium.Map(location=office_coord, zoom_start=11)
    folium.Marker(
        office_coord,
        icon=folium.Icon(color='red', icon='home'),
        popup='Офис'
    ).add_to(m)

    colors = {1: 'blue', 2: 'green', 3: 'purple'}

    for manager_id, group in day_df.groupby('manager_id'):
        color = colors.get(manager_id, 'gray')
        coords = [office_coord]

        for _, r in group.sort_values('order_in_route').iterrows():
            p = points_df[points_df.point_id == r.point_id].iloc[0]
            coords.append((p.latitude, p.longitude))
            folium.CircleMarker(
                [p.latitude, p.longitude],
                radius=5,
                color=color,
                fill=True,
                popup=f"Менеджер {manager_id}, точка {r.point_id}"
            ).add_to(m)

        coords.append(office_coord)
        folium.PolyLine(coords, color=color).add_to(m)

    return m

### Итоговый пайплайн

In [7]:
# Загрузка данных
df_raw = pd.read_csv(DATA_PATH)
df = validate_points(df_raw)

# Назначаем менеджеров
df = assign_managers(df, N_MANAGERS, RANDOM_SEED)

# Офис
office_lat, office_lon = geocode_address(OFFICE_ADDRESS)
office = (office_lat, office_lon)

# Рабочие дни
work_days = get_working_days(YEAR, MONTH)

# Разворачивание посещений
visits_flat = expand_visits_flat(df)

schedule_base, unserved = assign_visits_to_days(
    visits_flat,
    work_days,
    MAX_POINTS_PER_DAY
)

# Кластеризация + маршрутизация
result = []
for (manager_id, day), group in schedule_base.groupby(['manager_id', 'visit_day']):
    clustered = cluster_points(group, MAX_POINTS_PER_DAY, RANDOM_SEED)
    for cluster_id, cl in clustered.groupby('cluster_id'):
        routed = build_route(cl, office)
        routed['manager_id'] = manager_id
        routed['visit_day'] = day
        routed['cluster_id'] = cluster_id
        result.append(routed)

schedule = pd.concat(result)

### Экспорт в Excel

In [8]:
# Лист 1: детальный план
sheet1 = schedule[['point_id', 'visit_day', 'manager_id', 'cluster_id', 'order_in_route']]

# Лист 2: план по дням    
summary = []
for (day, manager), grp in schedule.groupby(['visit_day', 'manager_id']):
    coords = [office] + [
        (df[df.point_id == pid].latitude.values[0],
         df[df.point_id == pid].longitude.values[0])
        for pid in grp.sort_values('order_in_route').point_id
    ] + [office]

    travel_sec = osrm_route_duration(coords)

    service_min = df[df.point_id.isin(grp.point_id)].service_time_min.sum()

    if travel_sec is None:
        travel_min = np.nan
    else:
        travel_min = travel_sec / 60
    total_time = travel_min + service_min if not np.isnan(travel_min) else np.nan
    

    summary.append({
        'date': day,
        'manager_id': manager,
        'points': ', '.join(map(str, grp.point_id.tolist())),
        'points_count': len(grp),
        'total_time_min': round(travel_min + service_min, 1),
        'yandex_route': yandex_route_link(coords)
    })

sheet2 = pd.DataFrame(summary)

# Лист 3: необслуженные точки
served = set(schedule.point_id)

unserved = (
    df[~df.point_id.isin(served)]
    .loc[:, ['point_id', 'manager_id', 'latitude', 'longitude']]
    .copy()
)

unserved['dist_from_office_km'] = [
    geodesic(office, (lat, lon)).km
    for lat, lon in zip(unserved['latitude'].values, unserved['longitude'].values)
]

unserved = unserved.sort_values('dist_from_office_km', ascending=False)

with pd.ExcelWriter("routes_result.xlsx", engine="xlsxwriter") as writer:
    sheet1.to_excel(writer, sheet_name="Детальный план", index=False)
    sheet2.to_excel(writer, sheet_name="План по дням", index=False)
    unserved.to_excel(writer, sheet_name="Необслуженные точки", index=False)

In [9]:
visualize_day(schedule, df, office, VIZUALIZATION_DATE)